In [1]:
import numpy as np
import pandas as pd
import sqlite3
import kaggle

from IPython.display import display

pd.options.display.max_rows = 200

In [3]:
kaggle.api.authenticate()

In [20]:
path = "../../../data/sql_data/"

In [21]:
kaggle.api.dataset_download_files(
    'hugomathien/soccer', 
    path=path, 
    unzip=True
)

In [22]:
ls ../../../data/sql_data

database.sqlite


In [23]:
database = path + 'database.sqlite'

In [24]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [25]:
conn = create_connection(database)

In [26]:
def make_metadata_df(con):
    cur = conn.cursor()
    tablename_statement = "SELECT * FROM sqlite_master WHERE type='table';"
    res = []
    cur.execute(tablename_statement)
    meta_cols = {row[0]: num for num, row in enumerate(cur.description)}
    for row in cur:
        table_name = row[meta_cols["name"]]
        print(table_name)
        cols_statement = f"SELECT * from {table_name} LIMIT 1"
        new_cur = conn.cursor()
        new_cur.execute(cols_statement)
        for row in new_cur.description:
            res.append(dict(table_name=table_name,
                            col=row[0]))
        new_cur.close()
    cur.close()
    df = pd.DataFrame(res)
    return df

In [27]:
meta_df = make_metadata_df(conn)

sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


In [28]:
# meta_df.query("table_name == 'Match'")["col"].unique()

In [29]:
meta_df.table_name.unique()

array(['sqlite_sequence', 'Player_Attributes', 'Player', 'Match',
       'League', 'Country', 'Team', 'Team_Attributes'], dtype=object)

In [30]:
statement = \
"""
SELECT  
    m.date,
    t.team_long_name AS opponent,
    CASE WHEN m.home_team_goal < m.away_team_goal THEN 'Barcelona win!'
        WHEN m.home_team_goal > m.away_team_goal THEN 'Barcelona loss :(' 
        ELSE 'Tie' END AS outcome
FROM Match AS m
-- Join team to match
LEFT JOIN Team AS t 
ON m.home_team_api_id = t.team_api_id
WHERE m.away_team_api_id = 8634;
"""

In [39]:
# pd.read_sql("select * from Match limit 2", con=conn).T

In [43]:
def peek_sql_res(statement, con=conn):
    res_df = pd.read_sql(statement, con=conn)
    print(f"shape: {res_df.shape}")
    display(res_df.head(10))

In [44]:
peek_sql_res(statement)

shape: (152, 3)


,date,opponent,outcome
0,2008-08-31 00:00:00,CD Numancia,Barcelona loss :(
1,2008-11-16 00:00:00,RC Recreativo,Barcelona win!
2,2008-11-29 00:00:00,Sevilla FC,Barcelona win!
3,2008-12-21 00:00:00,Villarreal CF,Barcelona win!
4,2009-01-11 00:00:00,CA Osasuna,Barcelona win!
5,2009-02-01 00:00:00,Racing Santander,Barcelona win!
6,2009-02-14 00:00:00,Real Betis Balompié,Tie
7,2009-03-01 00:00:00,Atlético Madrid,Barcelona loss :(
8,2009-03-15 00:00:00,UD Almería,Barcelona win!
9,2009-04-04 00:00:00,Real Valladolid,Barcelona win!
